In [2]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import f1_score, classification_report, log_loss
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from lightgbm import LGBMClassifier


In [3]:
df = pd.read_csv('../Datasets/cases/Sonar/Sonar.csv')
X = df.drop('Class', axis=1)
y = df['Class']
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)


In [4]:
rates = np.linspace(0.01, 0.8, 20)
n_est = np.arange(50, 200, 30)
depths = [None, 2, 3, 4, 5, 7]
scores = []

for r in tqdm(rates):
    for n in n_est:
        for d in depths:
            lgbm = LGBMClassifier(random_state=23, n_estimators=n, max_depth=d, learning_rate=r, verbose=-1)
            lgbm.fit(X_train, y_train)
            y_pred_proba = lgbm.predict_proba(X_test)
            y_pred = lgbm.predict(X_test)
            scores.append([r,n,d,log_loss(y_test, y_pred_proba), f1_score(y_test, y_pred)])

scores = pd.DataFrame(scores, columns=['learning rate', 'estimator', 'max depth', 'log loss', 'f1'])
scores.sort_values(by='log loss')

100%|██████████| 20/20 [00:26<00:00,  1.33s/it]


,learning rate,estimator,max depth,log loss,f1
123,0.176316,50,4.0,0.371837,0.800000
93,0.134737,50,4.0,0.383137,0.784314
64,0.093158,50,5.0,0.390546,0.784314
60,0.093158,50,NaN,0.390546,0.784314
65,0.093158,50,7.0,0.390546,0.784314
...,...,...,...,...,...
492,0.675263,110,NaN,1.155843,0.708333
497,0.675263,110,7.0,1.155843,0.708333
486,0.675263,80,NaN,1.155843,0.708333
508,0.675263,170,5.0,1.155843,0.708333


In [5]:
scores.sort_values('f1', ascending=False)

,learning rate,estimator,max depth,log loss,f1
360,0.508947,50,NaN,0.592189,0.840000
364,0.508947,50,5.0,0.592189,0.840000
365,0.508947,50,7.0,0.592189,0.840000
494,0.675263,110,3.0,0.729628,0.830189
482,0.675263,50,3.0,0.682384,0.830189
...,...,...,...,...,...
4,0.010000,50,5.0,0.584243,0.487805
3,0.010000,50,4.0,0.584243,0.487805
2,0.010000,50,3.0,0.584243,0.487805
0,0.010000,50,NaN,0.584243,0.487805
